# mr-ash example analysis
Here we analyze [GTEx based simulation data](MR-ASH-Simulation.html) using mr-ash implemented  in `varbvs`. The simulated data is stored in the same format as the [processed GTEx](../pipeline/Pipelines.html#Preprocessing) data; therefore the same procedure can be used directly for data analysis. The only difference is that simulated data do not have covariates.

All files are saved in HDF5 format on midway `project/compbio/internal_public_supp/GTEx7Toy`. In the toy data-set I selected 3 genes:

```
/chr4/ENSG00000145214
/chr18/ENSG00000264247
/chr19/ENSG00000267508
```
These should be used as `geno_table` variable in the code below. 

For the toy real data there are 53 groups in the HDF file. Their names are available via:

```bash
h5ls TY.expr.h5 
```
For the simulated data there is only one group called `simulated`, which is the data I'll use in the example below.

To load data:

In [1]:
# source("http://bioconductor.org/biocLite.R")
# biocLite("rhdf5")
library(rhdf5)
fpath = '/home/gaow/Documents/GTEx/ToyExample'
genotype_file = paste0(fpath, '/TY.genotype.h5')
expr_file = paste0(fpath, '/TY.expr_simulated.h5')
geno_table = '/chr4/ENSG00000145214'
expr_table = '/simulated'

load_data = function(genotype_file, expr_file, geno_table, expr_table) {
    geno <- h5read(genotype_file, geno_table)
    gdata = geno$block0_values
    colnames(gdata) = geno$axis1
    rownames(gdata) = geno$axis0
    expr <- h5read(expr_file, expr_table)
    edata = expr$block0_values
    colnames(edata) = expr$axis1
    rownames(edata) = expr$axis0
    edata = edata[, basename(geno_table)]
    return(list(X=gdata,y=edata))
}

dat = load_data(genotype_file, expr_file, geno_table, expr_table)

To analyze:

In [2]:
# library(devtools)
# install_github("pcarbo/varbvs",subdir = "varbvs-R")
X = as.matrix(dat$X)
storage.mode(X) <- "double"
y = as.vector(dat$y)
res = varbvs::varbvsmix(X, NULL, y, (c(0,1,9))^2)

Fitting variational approximation for linear regression model with
mixture-of-normals priors.
samples:      635    mixture component sd's:    1..9
variables:    7258   fit mixture variances:     no
covariates:   0      fit mixture weights:       yes
mixture size: 3      fit residual var. (sigma): yes
intercept:    yes    convergence tolerance      1.0e-04
       variational    max. --------- hyperparameters ---------
iter   lower bound  change   sigma  mixture sd's  mix. weights
 


To compare with simulation meta data:

In [3]:
meta = paste0(fpath, '/TY.meta_simulation.json')
# install.packages('rjson', repos = 'http://cloud.r-project.org')
meta <- rjson::fromJSON(file = meta)
str(meta)

List of 4
 $ pi   : num [1:3] 0.25 0.3 0.45
 $ pi0  : num 0.98
 $ sigma: num [1:3] 1 0.4 3
 $ beta :List of 3
  ..$ ENSG00000264247: num [1:9871] -5.36 0 0 0 0 ...
  ..$ ENSG00000267508: num [1:8911] -4.83 0 0 0 0 ...
  ..$ ENSG00000145214: num [1:7258] 4.07 0 0 0 0 ...
